In [232]:
from pydub import AudioSegment
from transformers import pipeline
import torch
import pandas as pd
from ipywidgets import widgets, Button, Output
from IPython.display import display
import tkinter as tk
from tkinter import scrolledtext
from tkinter import ttk  # Progressbar를 위한 모듈
import threading


In [2]:
whisper_small = pipeline('automatic-speech-recognition','openai/whisper-small', device='mps')
whisper_tiny = pipeline('automatic-speech-recognition','openai/whisper-tiny',device='mps')

In [3]:
# whisper_small("./tts.mp3", generate_kwargs = {"language":"korean"})

In [4]:
whisper_tiny("./tts.mp3", generate_kwargs = {"language":"korean"})

/opt/anaconda3/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed language=korean, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of language=korean.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


{'text': ' 안녕하세요 오늘은 2014년 9월 11일 수요일입니다.'}

---

In [253]:
qa_model = pipeline(task='table-question-answering', model='dsba-lab/koreapas-finetuned-korwikitq', device=0)
# qa_model = pipeline(task='question-answering', model='monologg/koelectra-base-v3-finetuned-korquad')

/opt/anaconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [254]:
file_path = './country.csv'
country = pd.read_csv(file_path,encoding='euc-kr')

In [255]:
country.head(3)

,한글국가명,영문국가명,국가코드(ISO 2자리),수도,인구,인구설명,면적,면적설명,언어,종교,민족,기후,건국
0,가나,Ghana,GH,아크라(Accra),32400000,"(22, EIU)",238537.0,(한반도 1.1배),"영어(공용어), 아산테어, 에웨어 등","기독교(71%), 이슬람교(17.6%)","아칸족(48%), 몰다그바니족(17%), 에웨족(14%), 가아단베족(7%), 구르...",NaN,1957.3.6.(영국에서 독립)
1,가봉,Gabon,GA,리브르빌(Libreville),2340000,(21 World Bank),267000.0,(한반도의 1.2배),"불어(공용어), Fang어","기독교(가톨릭 포함) 85%, 회교 9.8%, 토착종교","Fang, Echira, Adouma 등 40여 종족",NaN,1960.8.17. 프랑스로부터 독립
2,가이아나,Guyana,GY,조지타운(Georgetown),750000,(2019 CIA),214969.0,NaN,"영어(공용어), Creole(현지 토속어)","기독교 57%, 힌두교 33%, 회교 9%, 기타 1%","동인도계 39.8%, 흑인 29.2%, 혼혈 19.9%, 아메리카 인디안 10.5%",NaN,NaN


In [256]:
# country = country[['한글국가명','수도','언어','면적설명']]
country = country[['한글국가명','수도']]

In [257]:
country.rename(columns={'한글국가명':'나라'},inplace=True)

In [258]:
country.fillna('',inplace=True)

In [259]:
country.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 0 to 196
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   나라      197 non-null    object
 1   수도      197 non-null    object
dtypes: object(2)
memory usage: 3.2+ KB


In [260]:
country.sample(10)

,나라,수도
117,에리트레아,"아스마라 (Asmara, 인구 80만명)"
146,중앙아프리카,"방기(Bangui, 79.4만명)"
177,투르크메니스탄,"아시가바트(Ashgabat, 84.6만명)"
168,쿡제도,아바루아(Avarua)
132,우크라이나,"키이우(키예프) (297만명, ‘19.12)"
3,감비아,반줄(Banjul)
51,모잠비크,마푸투(Maputo)
143,적도기니,"말라보(Malabo, 21만명)"
58,미합중국,"워싱턴(Washington, D.C)"
19,니우에,알로피(Alofi)


In [261]:
country.columns

Index(['나라', '수도'], dtype='object')

In [262]:
# KoELECTRA 질문-응답 모델 로드
qa_model = pipeline("question-answering", model="monologg/koelectra-base-v3-finetuned-korquad",device=0)
# qa_model = pipeline("question-answering", model="monologg/distilkobert", device=0)

/opt/anaconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [264]:
text = ""
for index, row in country.iterrows():
    text += f"{row['나라']}의 수도는 {row['수도']}입니다"

# 질문과 문맥 설정
context = text  # 변환된 텍스트

In [266]:
# 질문을 처리하는 함수 정의
def ask_question_in_background():
    threading.Thread(target=ask_question).start()
    
def ask_question():
    query = question_input.get()
    if query:
        try:
            # 로딩바 시작
            progress_bar.start()
            progress_label.config(text="질문 처리 중...")

            # 결과 상자 초기화 및 상태 메시지 출력
            output_area.config(state=tk.NORMAL)
            output_area.delete(1.0, tk.END)
            output_area.insert(tk.INSERT, "질문 처리 중...\n")
            output_area.config(state=tk.DISABLED)

            # GUI 업데이트
            root.update_idletasks()

            # 모델에 질문과 문맥 입력
            answer = qa_model(question=query, context=context)

            # 답변 출력
            result = answer['answer']
        except Exception as e:
            result = f"오류가 발생했습니다: {str(e)}"
        finally:
            # 로딩바 중지 및 텍스트 변경
            progress_bar.stop()
            progress_label.config(text="처리 완료")

        # 결과를 출력하는 텍스트 상자에 삽입
        output_area.config(state=tk.NORMAL)  # 출력하기 전에 텍스트 상자를 활성화
        output_area.delete(1.0, tk.END)  # 기존 내용을 지우기
        output_area.insert(tk.INSERT, f"질문: {query}\n답변: {result}\n")
        output_area.config(state=tk.DISABLED)  # 다시 텍스트 상자를 비활성화 (편집 방지)
    else:
        output_area.config(state=tk.NORMAL)
        output_area.insert(tk.INSERT, "질문을 입력해주세요.\n")
        output_area.config(state=tk.DISABLED)

# Tkinter GUI 설정
root = tk.Tk()
root.title("질문-응답 시스템")
root.geometry("500x450")

# 질문 입력 레이블과 입력창
question_label = tk.Label(root, text="질문을 입력하세요:")
question_label.pack(pady=10)
question_input = tk.Entry(root, width=50)
question_input.pack(pady=10)

# 질문 실행 버튼
ask_button = tk.Button(root, text="질문 실행", command=ask_question_in_background)
ask_button.pack(pady=10)

# 로딩바 추가
progress_bar = ttk.Progressbar(root, orient="horizontal", length=300, mode="indeterminate")
progress_bar.pack(pady=10)

# 로딩 상태를 표시할 레이블
progress_label = tk.Label(root, text="")
progress_label.pack()

# 결과 출력 상자 (스크롤 가능)
output_area = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=60, height=10, state=tk.DISABLED)
output_area.pack(pady=20)

# GUI 메인 루프 실행
root.mainloop()

: 